In [3]:
import json
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import tiktoken
import os
import openai  # Ensure you have imported the OpenAI library
import numpy as np
import tqdm


## Load API

In [4]:
# Initialize the OpenAI client

os.environ["OPENAI_API_KEY"] = "sk-proj-Hn7dQ7fUcD7YmwqXnTHtT3BlbkFJ8hFelf870E8SDDcKnXcF"
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

## Using chatgpt LLM

In [5]:
def content_generator(prompt, temp):
    prompt = f"""
    {prompt}
"""
    try:
        response = client.chat.completions.create(
            # model="gpt-4-turbo",
            model='gpt-4o',
            max_tokens=1024,
            temperature=temp,
            messages=[
    {"role": "user", "content": prompt}
    ])
        answer = response.choices[0].message.content
        return answer
    except Exception as e:
        print(f"Error: {e}")
        return None

## Prompt

### 1. Zero-shot prompting

- Zero-shot prompting relies on the model's ability to understand the task without any examples. The prompt contains only the instruction or question without any prior examples.

#### With RAG

- SMS/messages

In [6]:
zs_prompt = """
Use the following pieces of context to answer the question at the end. 
**Example:**
- Expected Output: "Make your money go further. Enjoy preferential 
FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. 
Save on fees when you send money home."

requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12371
subject: Stand to win S$8,888 when you refer friends to XYZ
template_text: Last week to stand to win S$8,888 & an omakase dinner for 2. Refer overseas friends to XYZ & earn up to $210. Get 1 lucky draw chance for every 5 who open accounts and deposit S$1k. Help them prep! They can open bank accounts remotely prior to arriving in.

description: Savings promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000

Question: Generate a remittance promotion message from Singapore to Malaysia
"""

zs_resp = content_generator(zs_prompt, 0.6)

- Email messages

In [7]:
zs_prompt1 = """
Use the following pieces of context to answer the question at the end. 
**Example:**
- Expected Output: "Make your money go further. Enjoy preferential 
FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. 
Save on fees when you send money home."
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12351
subject: Exclusive Benefits Await
template_text: Dear Lily, Step up your spending experience and apply for the XYZ Saver's Credit Card now. As our esteemed customer, you will enjoy $600 gift! Apply now to start your journey of privileges.  XYZ Team
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12435
subject: <ADV> Earn up to S$1,600 by referring XYZ to your family and friends
template_text: Dear Valued Client,
Love banking with us? Spread the word and earn S$80 cashback for every successful referral for a participating product. The more you refer, the more you earn, all the way up to S$1,600 cashback! T&Cs apply.
Refer and earn today by heading to: xyz.com/sg/refer 
Or look out for a push notification invitation in your XYZ Mobile
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
Question: 
Write an email for XYZ Bank to promote their investment services, highlighting the advantages over competitors
"""

zs_resp1 = content_generator(zs_prompt1, 0.6)

#### without RAG

- SMS

In [8]:
zs_prompt_no_rag = """
Use the following pieces of context to answer the question at the end. 
Generate a remittance promotion message from Singapore to Malaysia.

"""

zs_resp_no_rag = content_generator(zs_prompt_no_rag, 0.6)

- email

In [9]:
zs_prompt_no_rag1 = """
Write an email for XYZ Bank to promote their investment services, highlighting the advantages over competitors
"""

zs_resp_no_rag1 = content_generator(zs_prompt_no_rag1, 0.6)

### Direct prompting
- This involves providing a straightforward instruction or question to the model. The prompt is clear and unambiguous, guiding the model to produce the expected output.

#### with RAG

- SMS

In [10]:
direct_prompt = """
Use the following pieces of context to answer the question at the end.
1. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
3. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) accounts. Rest assured that rates quoted on our website are guaranteed even during market off hours (including public holidays and weekends). Log in to XYZ Internet Banking or the XYZ Digital app to get started now! Terms and conditions apply.
description: Exchange Rate Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
5. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply.
Save on fees when you send money home

description: Remittance promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
**Example:**
- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more."
"""

direct_resp = content_generator(direct_prompt, 0.6)

- email

In [11]:
direct_prompt1 = """
Use the following pieces of context to answer the question at the end.
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12351
subject: Exclusive Benefits Await
template_text: Dear Lily, Step up your spending experience and apply for the XYZ Saver's Credit Card now. As our esteemed customer, you will enjoy $600 gift! Apply now to start your journey of privileges.  XYZ Team
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12379
subject: Exclusive Credit Card Offers at XYZ!
template_text: Dear Customer, hello! Apply for our exclusive credit card now and enjoy up to 5% cashback on your monthly purchases, plus generous points that you can redeem for amazing rewards. Click www.xyz.com to apply and start enjoying these benefits today!
description: Credit Card Promotion
platform: Mobile
requirements: Customers between 18 to 80 years old; Minimum monthly salary of S$3000
5. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
Question: Compose an email for XYZ Bank that encourages recipients to explore their investment options.
"""

direct_resp1 = content_generator(direct_prompt1, 0.6)

#### without RAG

- SMS

In [12]:
direct_prompt_no_rag = """
Generate a remittance promotion message with the subject: XYZ Remit: Your Gateway to China
"""
direct_resp_no_rag = content_generator(direct_prompt_no_rag, 0.6)

- email

In [13]:
direct_prompt_no_rag1 = """
Compose an email for XYZ Bank that encourages recipients to explore their investment options.

"""
direct_resp_no_rag1 = content_generator(direct_prompt_no_rag1, 0.6)

### Few-shot prompting
- In few-shot prompting, the model is given a few examples of the task it is supposed to perform before the actual prompt. This helps the model understand the task better and produce more accurate results.

#### with RAG

- sms

In [14]:
fs_prompt = """Generate remittance promotion messages for the following subjects: 
1. XYZ Remit: Your Gateway to China-> 
Enjoy competitive rates and $0 transfer fees when you send money to China 
with XYZ Remit! Tap to find out more.
2. Send money to Malaysia at preferential rates. No cable/commission fees charged!
-> Make your money go further. Enjoy preferential FX rates when 
you make online transfers from your XYZ Singapore (SGD) account 
to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. 
Save on fees when you send money home. Fulfill your global needs when you remit with XYZ


**Example:**
- Expected Output: "Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply. Remit now"
1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply.
Save on fees when you send money home

description: Remittance promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
3. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) accounts. Rest assured that rates quoted on our website are guaranteed even during market off hours (including public holidays and weekends). Log in to XYZ Internet Banking or the XYZ Digital app to get started now! Terms and conditions apply.
description: Exchange Rate Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
5. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
"""

fs_resp = content_generator(fs_prompt, 0.6)

- email

In [15]:
fs_prompt1 = """
Use the following pieces of context to answer the question at the end. 
Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.

You may refer to the below past campaigns for your response:
1. template_id: 12345
subject: Enjoy portfolio promotion now!
template_text: Cushion the impact of market fluctuatiions by investing a fixed amount regularly! Simply set up a recurring top up and dollar-cost average with xyzPortfolio today!
description: Investment Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
2. template_id: 12374
subject: Enjoy potential capital growth and passive monthly income. Invest in a globally diversified fund that is exclusive to XYZ!
template_text: Limited tranche! Get potential monthly dividends and capital growth. Invest in XYZ's new globally diversified fund. This advertisement has not been reviewed by the Monetary Authority of Singapore. Investments are subject to investment risks, including the possible loss of the principal amount invested. Explore this option before the tranche is fully subscribed

description: Investment Promotion
platform: Mobile
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$15000
3. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12357
subject: Dedicated Investment Advisory
template_text: Looking for more investment opportunities? Gain access to discretionary portfolio management and dedicated advisory with XYZ high networth client services. Start a conversation with us today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial potential by starting a plan today.
Are you on track to meet your wealth goals?
Manage, grow and protect your wealth for generations to come with our XYZ Wealth Select advisory approach.
Start planning with XYZ Goals Planner: xyz.com/sg/wealth/xyz-goals-planner
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
"""

fs_resp1 = content_generator(fs_prompt1, 0.6)

#### without RAG

- SMS

In [16]:
fs_prompt_no_rag = """
Generate remittance promotion messages for the following subjects: XYZ Remit: Your Gateway to China ' -> Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.' ‘Send money to Malaysia at preferential rates. No cable/commission fees charged!' -> Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. 'Fulfill your global needs when you remit with XYZ' ->"
- Expected Output: " Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply. Remit now
"""

fs_resp_no_rag = content_generator(fs_prompt_no_rag, 0.6)

- email

In [17]:
fs_prompt_no_rag1 = """
Given these examples of email marketing messages for bank services, 
write a new one for XYZ Bank’s investment services focusing 
on risk management and potential returns. 

Examples: 
#1. Cushion the impact of market fluctuatiions 
by investing a fixed amount regularly! Simply set up a 
recurring top up and dollar-cost average with xyzPortfolio 
today!

#2. Seeking hassle-free invsesting? Tap here to select 
your preferred review frequency, and we will tailor product recommendations 
to match your preference.

#3. Limited tranche! Get potential 
monthly dividends and capital growth. Invest in XYZ new 
globally diversified fund. This advertisement has not been 
reviewed by the Monetary Authority of Singapore. Investments 
are subject to investment risks, including the possible loss 
of the principal amount invested. Explore this option before 
the tranche is fully subscribed.
"""

fs_resp_no_rag1 = content_generator(fs_prompt_no_rag1, 0.6)

### Contextual prompting
- In contextual prompting, the model is given a context or background information before the actual prompt. This context helps the model understand the situation or topic better.

#### with RAG

- SMS

In [18]:
contextual_prompt = """
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. 
Commission fees will also be waived for these transfers. 
The promotion is subjected to T&Cs."
Generate a promotional message for cross border transfers with Bank XYZ.

- Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "

You may refer to past promotional campaigns below to craft your messages:
1. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) accounts. Rest assured that rates quoted on our website are guaranteed even during market off hours (including public holidays and weekends). Log in to XYZ Internet Banking or the XYZ Digital app to get started now! Terms and conditions apply.
description: Exchange Rate Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
2. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply.
Save on fees when you send money home

description: Remittance promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
3. template_id: 12423
subject: <ADV> Limited Time: Enjoy ZERO foreign exchange (FX) fees when you spend overseas with XYZ Credit Cards
template_text: What better way to travel – enjoy ZERO FX fees on your overseas spends (min. spend of S$300 equivalent) with any XYZ Credit Cards in Malaysia, Thailand and Japan from 15 March to 15 April 2024. 
To register, simply SMS SCFX to 77222.
Please visit our official website for offer details. Terms and Conditions apply.
description: Remittance promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
5. template_id: 12388
subject: Save Big with Our Auto Loan Special Rates!
template_text: Drive away with a great deal! Enjoy special rates on auto loans with xyz. Low interest and quick approval—apply today! Visit [bank website] or call 1234 5678 for more details.
description: Loan Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
 **End of Prompt**

"""
contextual_resp = content_generator(contextual_prompt, 0.6)

- email

In [19]:
contextual_prompt1 = """
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free

You may refer to past promotional campaigns below to craft your messages:
1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12352
subject: Retirement Planning Services
template_text: Dear Alice, Prepare for the future with XYZ Bank's professional retirement planning services to help you achieve financial freedom. Our retirement planning advisors will work with you one-on-one to ensure a worry-free retirement life. The XYZ Bank Team
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12388
subject: Save Big with Our Auto Loan Special Rates!
template_text: Drive away with a great deal! Enjoy special rates on auto loans with xyz. Low interest and quick approval—apply today! Visit [bank website] or call 1234 5678 for more details.
description: Loan Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial potential by starting a plan today.
Are you on track to meet your wealth goals?
Manage, grow and protect your wealth for generations to come with our XYZ Wealth Select advisory approach.
Start planning with XYZ Goals Planner: xyz.com/sg/wealth/xyz-goals-planner
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12379
subject: Exclusive Credit Card Offers at XYZ!
template_text: Dear Customer, hello! Apply for our exclusive credit card now and enjoy up to 5% cashback on your monthly purchases, plus generous points that you can redeem for amazing rewards. Click www.xyz.com to apply and start enjoying these benefits today!
description: Credit Card Promotion
platform: Mobile
requirements: Customers between 18 to 80 years old; Minimum monthly salary of S$3000
 **End of Prompt**
"""
contextual_resp1 = content_generator(contextual_prompt1, 0.6)

#### without RAG

- SMS

In [20]:
contextual_prompt_no_rag = """
Context: "Bank XYZ is offering excellent FX rates for online transfers between Singapore and Malaysia accounts, from SGD to MYR. Commission fees will also be waived for these transfers. The promotion is subjected to T&Cs."
Prompt: "Generate a promotional message for cross border transfers with Bank XYZ."
Expected Output: "Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home. "
"""
contextual_resp_no_rag = content_generator(contextual_prompt_no_rag, 0.6)

- Email

In [45]:
contextual_prompt_no_rag1 = """
Use the following pieces of context to answer the below question/request. 
Context: "Bank XYZ is offering wealth planning services. It helps to promote tax savings and users can be worry-free"
"""
contextual_resp_no_rag1 = content_generator(contextual_prompt_no_rag1, 0.6)

### Role-playing/Persona-based prompt
- This technique involves assigning a specific role or persona to the model, influencing the style and content of the response.

#### with RAG

- SMS

In [21]:
persona_prompt = """
 
Use the following pieces of context to answer the below question/request. 
You are a marketing agent for Bank XYZ. 
Generate a message to promote the bank’s remittance product which 
offers no transfer fees to China.

- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
3. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
4. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12435
subject: <ADV> Earn up to S$1,600 by referring XYZ to your family and friends
template_text: Dear Valued Client,
Love banking with us? Spread the word and earn S$80 cashback for every successful referral for a participating product. The more you refer, the more you earn, all the way up to S$1,600 cashback! T&Cs apply.
Refer and earn today by heading to: xyz.com/sg/refer 
Or look out for a push notification invitation in your XYZ Mobile
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
"""

persona_resp = content_generator(persona_prompt, 0.6)

- email

In [22]:
persona_prompt1 = """
Use the following pieces of context to answer the below question/request. 
As a young professional looking to secure your 
financial future, XYZ Bank’s investment options offer the 
perfect balance of risk and reward. Craft an email targeting 
this persona.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial potential by starting a plan today.
Are you on track to meet your wealth goals?
Manage, grow and protect your wealth for generations to come with our XYZ Wealth Select advisory approach.
Start planning with XYZ Goals Planner: xyz.com/sg/wealth/xyz-goals-planner
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12443
subject: <ADV> What’s new on XYZ Goals Planner: You can now access and complete your investment profile for personalised insights
template_text: Dear Valued Client,
XYZ Goals Planner is our platform that helps you lay out your financial plan to achieve your goals, whether it is saving for retirement, a new home or pursuing the things you love. The insights from the financial plan combined with your investment profile will enable us to provide personalised wealth solutions to achieve your goals.
Start your journey by completing your financial plan and your investment profile today!
To find out more about XYZ Goals Planner, visit our website and search for "Goals Planner".
description: Wealth planning promotion
platform: Email
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$5000
"""

persona_resp1 = content_generator(persona_prompt1, 0.6)

#### without RAG

- SMS

In [23]:
persona_prompt_no_rag = """
**Example:**
- Prompt: "You are a marketing agent for Bank XYZ. Generate a message to promote the bank’s remittance product which offers no transfer fees to China."
- Expected Output: "Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.”
"""
persona_resp_no_rag = content_generator(persona_prompt_no_rag, 0.6)

- email

In [24]:
persona_prompt_no_rag1 = """
As a young professional looking to secure your financial future, XYZ Bank’s investment options offer the perfect balance of risk and reward. Craft an email targeting this persona.
"""
persona_resp_no_rag1 = content_generator(persona_prompt_no_rag1, 0.6)

### Completion prompt
- Here, the model is given an incomplete sentence or story and asked to complete it. This technique is often used in creative writing and story generation.

#### with RAG

- SMS

In [25]:
completion_prompt = """
Use the following pieces of context to answer the below question/request. 
Make your money go further. Enjoy preferential FX rates …
Expected output: when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home.

You can use the below context to guide your response:
1. template_id: 12423
subject: <ADV> Limited Time: Enjoy ZERO foreign exchange (FX) fees when you spend overseas with XYZ Credit Cards
template_text: What better way to travel – enjoy ZERO FX fees on your overseas spends (min. spend of S$300 equivalent) with any XYZ Credit Cards in Malaysia, Thailand and Japan from 15 March to 15 April 2024. 
To register, simply SMS SCFX to 77222.
Please visit our official website for offer details. Terms and Conditions apply.
description: Remittance promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
2. template_id: 12420
subject: <ADV> 0 Foreign Transaction Fee and Uncapped 1.5% Cashback with XYZ Simply Cash Credit Card
template_text: Get the most out of your travel spends with 1.‌5% uncapped cashback and for a limited time only, 0 foreign transaction fee with XYZ Simply Cash Credit Card.
Simply charge S$2,500 or more to your card from now till 3‌1 Ma‌y 20‌24 to unlock 0 foreign transaction fee^ for all your overseas spends in Ju‌ne 20‌2‌4.
To register, SMS FXSC to 772‌22. This offer is limited to the first 5,000 cardholders.
Please visit XYZ Bank Singapore website and search for "The Good Life Promotions" for offer details. For the full spends promotion details, please visit our official website. T&Cs apply.
description: Remittance promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12394
subject: <ADV> Send money from Singapore to Malaysia at preferential rates!
template_text: From now till 30 April 2024, enjoy preferential exchange rates and avoid cable/commission fees when you transfer funds from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) accounts. Rest assured that rates quoted on our website are guaranteed even during market off hours (including public holidays and weekends). Log in to XYZ Internet Banking or the XYZ Digital app to get started now! Terms and conditions apply.
description: Exchange Rate Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
4. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply.
Save on fees when you send money home

description: Remittance promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
5. template_id: 12439
subject: <ADV> Earn higher interest of up to 7%p.a. on your USD with USD$aver
template_text: Dear Valued Client,
We are excited to share that our USD$aver now comes with a higher interest rate of up to 7% p.a. to help you grow your USD deposits with no lock-in period.
Visit USD$aver webpage for more details.
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
"""
completion_resp = content_generator(completion_prompt, 0.6)

- email

In [26]:
completion_prompt1 = """
Use the following pieces of context to answer the below question/request. 
XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...
Complete the request above with a promotional email message.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12400
subject: <ADV> Earn bigger returns with up to 4.65% a year when you credit your salary, save and spend on an XYZ Account
template_text: From up to 4.05% a year, we’ve increased our interest rate to up to 4.65% when you credit your salary, save and spend on your XYZ Account. Earn bigger, better gains of additional 3% a year when you invest and insure! So if you’re earning less interest elsewhere, now’s the time to consolidate your funds with the XYZ Account to maximise your savings. Terms and conditions apply.
description: Credit Card Promotion
platform: Mobile
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12354
subject: Plan with XYZ
template_text: Seeking hassle-free invsesting? Tap here to select your preferred review frequency, and we'll tailor product recommendations to match your preference.
description: Investment Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
4. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12379
subject: Exclusive Credit Card Offers at XYZ!
template_text: Dear Customer, hello! Apply for our exclusive credit card now and enjoy up to 5% cashback on your monthly purchases, plus generous points that you can redeem for amazing rewards. Click www.xyz.com to apply and start enjoying these benefits today!
description: Credit Card Promotion
platform: Mobile
"""

completion_resp1 = content_generator(completion_prompt1, 0.6)

#### without RAG

- SMS

In [27]:
completion_prompt_no_rag = """
Use the following pieces of context to answer the below question/request. 
Make your money go further. Enjoy preferential FX rates …
Expected output: when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply. Save on fees when you send money home.
"""
completion_resp_no_rag = content_generator(completion_prompt_no_rag, 0.6)

- email

In [28]:
completion_prompt_no_rag1 = """
XYZ Bank’s new investment plan is designed to offer the best returns on the market. This plan includes...
"""
completion_resp_no_rag1 = content_generator(completion_prompt_no_rag1, 0.6)

### Guided prompt

#### with RAG
- sms

In [29]:
guided_prompt = """
Use the following pieces of context to answer the below question/request. 

1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
4. template_id: 12371
subject: Stand to win S$8,888 when you refer friends to XYZ
template_text: Last week to stand to win S$8,888 & an omakase dinner for 2. Refer overseas friends to XYZ & earn up to $210. Get 1 lucky draw chance for every 5 who open accounts and deposit S$1k. Help them prep! They can open bank accounts remotely prior to arriving in.

description: Savings promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$3000
5. template_id: 12435
subject: <ADV> Earn up to S$1,600 by referring XYZ to your family and friends
template_text: Dear Valued Client,
Love banking with us? Spread the word and earn S$80 cashback for every successful referral for a participating product. The more you refer, the more you earn, all the way up to S$1,600 cashback! T&Cs apply.
Refer and earn today by heading to: xyz.com/sg/refer 
Or look out for a push notification invitation in your XYZ Mobile
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000

Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.
"""

guided_resp = content_generator(guided_prompt, 0.6)

- Email

In [30]:
guided_prompt1 = """
Use the following pieces of context to answer the below question/request. 

Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12400
subject: <ADV> Earn bigger returns with up to 4.65% a year when you credit your salary, save and spend on an XYZ Account
template_text: From up to 4.05% a year, we’ve increased our interest rate to up to 4.65% when you credit your salary, save and spend on your XYZ Account. Earn bigger, better gains of additional 3% a year when you invest and insure! So if you’re earning less interest elsewhere, now’s the time to consolidate your funds with the XYZ Account to maximise your savings. Terms and conditions apply.
description: Credit Card Promotion
platform: Mobile
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12354
subject: Plan with XYZ
template_text: Seeking hassle-free invsesting? Tap here to select your preferred review frequency, and we'll tailor product recommendations to match your preference.
description: Investment Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
4. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12379
subject: Exclusive Credit Card Offers at XYZ!
template_text: Dear Customer, hello! Apply for our exclusive credit card now and enjoy up to 5% cashback on your monthly purchases, plus generous points that you can redeem for amazing rewards. Click www.xyz.com to apply and start enjoying these benefits today!
description: Credit Card Promotion
platform: Mobile
requirements: Customers between 18 to 80 years old; Minimum monthly salary of S$3000
"""

guided_resp1 = content_generator(guided_prompt1, 0.6)

#### without RAG

- Messages

In [49]:
guided_prompt_no_rag = """Generate a compelling marketing message for XYZ Bank that highlights the benefits of their new remittance promotions for customers looking to remit money overseas.
Focus on the features and potential rebates customers will get to enjoy.

"""

guided_resp_no_rag = content_generator(guided_prompt_no_rag, 0.6)

- Email

In [47]:
guided_prompt_no_rag1 = """Generate a compelling email marketing message for XYZ Bank that highlights the benefits of their new investment plan for customers looking to maximize their savings. 
Focus on the features and potential returns of the plan.
"""

guided_resp_no_rag1 = content_generator(guided_prompt_no_rag1, 0.6)

### Hypothetical prompt

#### with RAG

- sms

In [33]:
hypothetical_prompt = """
 
Use the following pieces of context to answer the below question/request. 

Imagine a customer looking to remit their money overseas monthly to his/her family after receiving monthly salaries. Write a marketing message for XYZ Bank that convinces them to use XYZ’s remittance services by illustrating the benefits of remitting money monthly with XYZ.
1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
4. template_id: 12388
subject: Save Big with Our Auto Loan Special Rates!
template_text: Drive away with a great deal! Enjoy special rates on auto loans with xyz. Low interest and quick approval—apply today! Visit [bank website] or call 1234 5678 for more details.
description: Loan Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12381
subject: Unlock Your Financial Potential with Our New Savings Account!
template_text: Open a new savings account with XYZ today and enjoy competitive interest rates and easy online management. Secure your financial future with us! Visit www.xyz.com or call 1234 5678 for more information.
description: Savings Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
"""

hypothetical_resp = content_generator(hypothetical_prompt, 0.6)

- Email

In [34]:
hypothetical_prompt1 = """
 
Use the following pieces of context to answer the below question/request. 

Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security.

1. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
2. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12351
subject: Exclusive Benefits Await
template_text: Dear Lily, Step up your spending experience and apply for the XYZ Saver's Credit Card now. As our esteemed customer, you will enjoy $600 gift! Apply now to start your journey of privileges.  XYZ Team
description: Credit Card Promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
4. template_id: 12443
subject: <ADV> What’s new on XYZ Goals Planner: You can now access and complete your investment profile for personalised insights
template_text: Dear Valued Client,
XYZ Goals Planner is our platform that helps you lay out your financial plan to achieve your goals, whether it is saving for retirement, a new home or pursuing the things you love. The insights from the financial plan combined with your investment profile will enable us to provide personalised wealth solutions to achieve your goals.
Start your journey by completing your financial plan and your investment profile today!
To find out more about XYZ Goals Planner, visit our website and search for "Goals Planner".
description: Wealth planning promotion
platform: Email
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial potential by starting a plan today.
Are you on track to meet your wealth goals?
Manage, grow and protect your wealth for generations to come with our XYZ Wealth Select advisory approach.
Start planning with XYZ Goals Planner: xyz.com/sg/wealth/xyz-goals-planner
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
"""

hypothetical_resp1 = content_generator(hypothetical_prompt1, 0.6)

#### without RAG

- Messages

In [35]:
hypothetical_prompt_no_rag = """Use the following pieces of context to answer the question at the end. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security."""

hypothetical_resp_no_rag = content_generator(hypothetical_prompt_no_rag, 0.6)

- Email

In [36]:
hypothetical_prompt_no_rag1 = """Use the following pieces of context to answer the question at the end. 
Imagine a customer looking to invest their annual bonus. 
Write an email for XYZ Bank that convinces them to choose 
XYZ’s investment services by explaining how it aligns with 
their goal of future financial security."""

hypothetical_resp_no_rag1 = content_generator(hypothetical_prompt_no_rag1, 0.6)

## Chain of thought prompting

### with RAG

- sms

In [37]:
cot_prompt = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs remittance service emphasizing its exclusive promotion. Highlight the service low charges and extensive countries coverage.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.

The context is provided below for your guidance:
1. template_id: 12414
subject: <ADV> Send money overseas to your loved ones at $0 transfer charges with XYZ Remit
template_text: Enjoy $0 transfer charges to more than 50 countries globally
Sending money overseas has never been easier. With XYZ Remit, you can now enjoy the convenience of sending money to your loved ones abroad without worrying about high fees and complicated paperwork.
Check out our rates on XYZ Mobile with a few clicks when you send money home to the following markets and more.
For more details, visit our website > Services > XYZ Remit.  
description: Remittance promotion
platform: Email
requirements: Customers between 18 to 80 years old
2. template_id: 12376
subject: Fulfill your global needs when you remit with XYZ
template_text: Benefit from no cable and commission fees when you make overseas remittances online in 19 currencies, including USD, AUD and EUR. Enjoy this perk when you send money to loved ones, pay tuition or property fees abroad, and more. Agent fees, if applicable, will still be charged. Till 31 Dec 2024. T&Cs apply.
Remit now

description: Remittance promotion
platform: Mobile
requirements: Customers between 25 to 80 years old; Minimum monthly salary of S$5000
3. template_id: 12353
subject: XYZ Remit: Your Gateway to China
template_text: Enjoy competitive rates and $0 transfer fees when you send money to China with XYZ Remit! Tap to find out more.
description: Remittance promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
4. template_id: 12370
subject: Send money to Malaysia at preferential rates. No cable/commission fees charged!
template_text: Make your money go further. Enjoy preferential FX rates when you make online transfers from your XYZ Singapore (SGD) account to any XYZ Malaysia (MYR) one. What is more, we will waive all cable/commission fees; agent fees, if any, will still be charged. T&Cs apply.
Save on fees when you send money home

description: Remittance promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$2000
5. template_id: 12388
subject: Save Big with Our Auto Loan Special Rates!
template_text: Drive away with a great deal! Enjoy special rates on auto loans with xyz. Low interest and quick approval—apply today! Visit [bank website] or call 1234 5678 for more details.
description: Loan Promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
 **End of Prompt**
{result}
"""

cot_resp = content_generator(cot_prompt, 0.6)

- email

In [38]:
cot_prompt1 = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs wealth planning services. Highlight its benefits and the high returns that the service is able to provide.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.

The context is provided below for your guidance:
1. template_id: 12387
subject: Invest in Your Future with Our Wealth Management Services!
template_text: Invest in your future with our personalized wealth management services. Get expert advice and explore a range of investment opportunities. Contact our advisors at1234 5678 or visit www.xyz.com to get started.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
2. template_id: 12358
subject: Bespoke Wealth Planning
template_text: Will your children have enough to chart their legacy? Let XYZ High Networth Client services help with succession planning and professional advisory. Start a conversation with us today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 20 to 80 years old; Minimum monthly salary of S$15000
3. template_id: 12385
subject: Plan Your Future with Our Retirement Savings Plan!
template_text: Dear Customer, secure your retirement with our comprehensive savings plan! Enjoy potential tax advantages and plan for a worry-free future. Learn more at www.xyz.com or call 1234 5678 today.
description: Wealth planning promotion
platform: Mobile
requirements: Customers between 18 to 80 years old
4. template_id: 12441
subject: <ADV> Your guide to financial planning in your 20s
template_text: Dear Valued Client,
Being in your 20s means time is on your side. While you've just stepped into the adulting world, take advantage of the magic of compounding and watch your money grow over time. Unleash your full financial potential by starting a plan today.
Are you on track to meet your wealth goals?
Manage, grow and protect your wealth for generations to come with our XYZ Wealth Select advisory approach.
Start planning with XYZ Goals Planner: xyz.com/sg/wealth/xyz-goals-planner
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
5. template_id: 12352
subject: Retirement Planning Services
template_text: Dear Alice, Prepare for the future with XYZ Bank's professional retirement planning services to help you achieve financial freedom. Our retirement planning advisors will work with you one-on-one to ensure a worry-free retirement life. The XYZ Bank Team
description: Wealth planning promotion
platform: Email
requirements: Customers between 30 to 80 years old; Minimum monthly salary of S$5000
 **End of Prompt**
{result}
"""

cot_resp1 = content_generator(cot_prompt1, 0.6)

### without RAG

- sms

In [39]:
cot_prompt_no_rag = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs remittance service emphasizing its exclusive promotion. Highlight the service low charges and extensive countries coverage.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
 **End of Prompt**
{result}
"""

cot_resp_no_rag = content_generator(cot_prompt_no_rag, 0.6)

- emails

In [40]:
cot_prompt_no_rag1 = """
Guidelines:
Designing beaviourally-informed comms
GRAB your recipients' attention by following these steps.
G - Goal of the comms: State it
R - Recipient: understand them
A - Audit: Conduct a behavioural audit of comms
B - behavioural principles: apply relevant one(s) to enhance comms

1. Goal of the comms: define the goal of the comms. This is the desired action we want receipients to take upon reading the comms.
Consider the following guiding questions when defining the goal of the comms:
- What is the key behaviour we want the recipient to take?
- What are the steps that the recipient has to take to perform the behaviour?

Crafting a goal statement
The goal of the comms is for <who> to <perform what action> by <how>.

2. Recipient - Understand the recipient
Only by understanding the jobs-to-be-done, needs, and pain points of the target audience can we tailor our comms to achieve the desired response.

Consider the following guiding questions:
- Target audience
Who needs to perform the target behaviour? e.g. customers with outstanding loans
- What are the demographics of the target audience? e.g. Gender, age, ethnicity, income, needs and etc
- Familiarity of task: are they familiar with the action required? e.g. have they done it before? will they find it difficult to do?
Benefits and costs
What are the benefits to the recipient for taking action? Are the benefits felt immediately or in the future? What are the costs to the recipient for taking the action?
- Optimising comms (good to consider): What is the best channel to maximise engagement with the target audience? e.g. SMS, email, push note, etc. What are their preferences and needs? 
What is the optimal time to send or receive? (What their habits/preferences - e.g. AM/PM weekends, payday etc). Who might the recipient respond to best? (Who is the best messenger to send the comms?)

3. Audit - conduct a behavioral audit of the current comms. After defining the goal of the comms and understanding the recipient better, conduct a behavoural audit of the current comms to uncover barriers that can reduce the impact of the comms.
Step 1: Get a sample of the current comms
Step 2: Audit the comms by reading it from the recipient's point of view

Consider the following guiding questions as you audit the comms to identify the areas that can be improved
- Language: is the language difficult to understand? Does it contain technical jargon? Does it contain knowledge that can be interpreted inaccurately? Does it require any baseline or prior knowledge?

- Content presentation:
-> Is the order logical?
-> Is the key info communicated upfront?

- Call to action: Are the benefits of taking action and/or consequences of not taking action clear? (i.e. do we answer the recipient's question of "what is in it for me?") Is there a clear call to action (i.e. can the recipient understand what is being asked of them and within the first few lines)? Are there clear steps which the recipient can follow? Are the steps to be taken perceived as easy?

- Personalization and tone: Is personalized content used? Is the tone appropriate (e.g. direct, authoritarian, positive, friendly, empowering, etc)?
- Messenger: Is the sender someone who the recipient views as influential in relation to the message?

- Management of expectations: Does the recipient know what happens next? Are deadlines clear, if any? If there is nothing for them to do, did we let them know?

4. Behavioural principles - Apply relevant ones to enhance comms
Based on guiding questions in step 3, apply to the context on hand.

Instructions: Craft a promotional message for the XYZs wealth planning services. Highlight its benefits and the high returns that the service is able to provide.

You are a marketing expert from XYZ who is helpful and technical. You will use the following thought process to think, question and answer step by step. Do not repeat the instructions in the following steps in your response as they are meant to be hidden from the user.

Step 1: Analyze and understand the instructions provided to generate the required marketing content as provided in the 'instructions' section. When doing this step, pelase use the info provided in the guidelines to craft the content. Answer all guiding questions for the first two GRAB components (G and R) by listing down ALL guiding questions, and the answer to them.
If you feel that any information is missing, please ask for more information in the response for this step. 

Step 2: Using the demographics identified in step 1, generate threee distinct subsegments of customers and suggest how to appeal to the needs of these customers. Hwoever, ensure that you take into account the limitations from the instructions.

Step 3: Design and generate personalized marketing contents to target the subsegments identified in step 2. Please try to be as verbose as possible. 

Step 4: Analyze each of the generated marketing content in step 3 with the next 2 GRAB components (A and B). Please provide a concise summary on how each of the generated marketing content follows the guidelines provided. Separate the points into common and unique points.
 **End of Prompt**
{result}
"""

cot_resp_no_rag1 = content_generator(cot_prompt_no_rag1, 0.6)

## Compile results

In [50]:
dict_results = {
    "prompt":[zs_prompt, zs_prompt1, direct_prompt, direct_prompt1, fs_prompt, fs_prompt1, cot_prompt, cot_prompt1, contextual_prompt, contextual_prompt1, persona_prompt, persona_prompt1, completion_prompt, completion_prompt1, guided_prompt, guided_prompt1, hypothetical_prompt, hypothetical_prompt1],
    "prompt_type":['Zero-shot prompting', 'Zero-shot prompting', 'Direct prompting', 'Direct prompting', 'Few-shot prompting', 'Few-shot prompting', 'COT prompting', 'COT prompting', 'Contextual prompting', 'Contextual prompting', 'Persona prompting', 'Persona prompting', 'Completion prompting', 'Completion prompting', 'Guided prompting', 'Guided prompting', 'Hypothetical prompting', 'Hypothetical prompting'],
    "response(with RAG)":[zs_resp, zs_resp1, direct_resp, direct_resp1, fs_resp, fs_resp1, cot_resp, cot_resp1, contextual_resp, contextual_resp1, persona_resp, persona_resp1, completion_resp, completion_resp1, guided_resp, guided_resp1, hypothetical_resp, hypothetical_resp1],
    "response(without RAG)":[zs_resp_no_rag, zs_resp_no_rag1, direct_resp_no_rag, direct_resp_no_rag1, fs_resp_no_rag, fs_resp_no_rag1, cot_resp_no_rag, cot_resp_no_rag1, contextual_resp_no_rag, contextual_resp_no_rag1, persona_resp_no_rag, persona_resp_no_rag1, completion_resp_no_rag, completion_resp_no_rag1, guided_resp_no_rag, guided_resp_no_rag1, hypothetical_resp_no_rag, hypothetical_resp_no_rag1]
}

results = pd.DataFrame(dict_results)
results

prompt             prompt_type  \
0   \nUse the following pieces of context to answe...     Zero-shot prompting   
1   \nUse the following pieces of context to answe...     Zero-shot prompting   
2   \nUse the following pieces of context to answe...        Direct prompting   
3   \nUse the following pieces of context to answe...        Direct prompting   
4   Generate remittance promotion messages for the...      Few-shot prompting   
5   \nUse the following pieces of context to answe...      Few-shot prompting   
6   \nGuidelines:\nDesigning beaviourally-informed...           COT prompting   
7   \nGuidelines:\nDesigning beaviourally-informed...           COT prompting   
8   \nUse the following pieces of context to answe...    Contextual prompting   
9   \nUse the following pieces of context to answe...    Contextual prompting   
10  \n \nUse the following pieces of context to an...       Persona prompting   
11  \nUse the following pieces of context to answe...       Persona prompting   
12  \nUse the following pieces of context to answe...    Completion prompting   
13  \nUse the following pieces of context to answe...    Completion prompting   
14  \nUse the following pieces of context to answe...        Guided prompting   
15  \nUse the following pieces of context to answe...        Guided prompting   
16  \n \nUse the following pieces of context to an...  Hypothetical prompting   
17  \n \nUse the following pieces of context to an...  Hypothetical prompting   

                                   response(with RAG)  \
0   "Make your money go further. Enjoy preferentia...   
1   **Subject: Unlock Superior Investment Opportun...   
2   - Expected Output: "Enjoy $0 transfer charges ...   
3   Subject: Explore Your Investment Options with ...   
4   Here are the remittance promotion messages bas...   
5   **Subject:** Maximize Returns with Expert Risk...   
6   ### Step 1: Analyze and Understand the Instruc...   
7   ### Step 1: Analyze and Understand Instruction...   
8   **Subject:** <ADV> Send Money from Singapore t...   
9   Based on the provided context and past promoti...   
10  Certainly! Here is a promotional message for B...   
11  ### Subject: Invest in Your Future with XYZ Ba...   
12  Certainly! Here is a comprehensive and cohesiv...   
13  Subject: Plan Your Future with XYZ Bank’s New ...   
14  **Subject: <ADV> Seamless Overseas Remittances...   
15  **Subject: Maximize Your Savings with XYZ Bank...   
16  **Subject: <ADV> Send Money Overseas to Your L...   
17  Subject: Invest in Your Future with Our Wealth...   

                                response(without RAG)  
0   Sure! Here's a remittance promotion message ta...  
1   Subject: Unlock Your Financial Future with XYZ...  
2   **Subject: XYZ Remit: Your Gateway to China**\...  
3   Subject: Unlock Your Financial Future with XYZ...  
4   ### Remittance Promotion Messages\n\n#### 1. X...  
5   Subject: Maximize Your Returns with XYZ Bank's...  
6   ### Step 1: Analyze and Understand Instruction...  
7   ### Step 1: Understanding the Instructions and...  
8   "Make your money go further with Bank XYZ! Enj...  
9   Bank XYZ is offering wealth planning services ...  
10  Sure, here’s a promotional message for Bank XY...  
11  Subject: Secure Your Financial Future with XYZ...  
12  Make your money go further. Enjoy preferential...  
13  ...a diversified portfolio that encompasses a ...  
14  🌍✨ **Send Money Overseas with Ease – XYZ Bank’...  
15  Subject: Unlock Your Financial Future with XYZ...  
16  Subject: Secure Your Future with XYZ Bank’s Ex...  
17  Subject: Secure Your Financial Future with XYZ...

## Export to CSV

In [52]:
results.to_csv('results_chatgpt_0626.csv', index=False)